<a href="https://colab.research.google.com/github/Fabiancaru/Advanced_Methods_Data_Analysis_II/blob/main/generacion_texto_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GENERACIÓN DE TEXTO CON REDES NEURONALES RECURRENTES

## 1 - El set de datos y el problema a resolver

Usaremos un texto en español, de libre acceso, y que contiene una serie de [cuentos de Edgar Allan Poe](https://www.gutenberg.org/cache/epub/46196/pg46196.txt).

La idea es crear un modelo de lenguaje a nivel de caracter: para cada predicción tomará 10 caracteres de entrada y generará 1 de salida:

![](https://drive.google.com/uc?export=view&id=1UqXixOnlp4AS7gL1EZiZW0J1qJ9ctg3B)

In [ ]:
# Las librerías a utilizar
import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN
from keras.models import Sequential
from google.colab import drive
import tensorflow as tf

In [ ]:
# Leer archivo
drive.mount('/gdrive')
ruta = '/gdrive/MyDrive/Colab Notebooks/01_2021-07-FundamentosDL/3.7-RNN-Practica1-GeneracionTextoConRNNs/cuentos_EAPoe.txt'

texto = open(ruta,'r',encoding='utf8').read()
chars = list(set(texto))
tam_vocab = len(chars)

print(texto)
print(chars)
print(tam_vocab)

## 2 - Creación del set de entrenamiento

In [ ]:
# Diccionarios para convertir de texto a índice numérico y viceversa
char2ix = {c: i for i, c in enumerate(chars)}
ix2char = {i: c for i, c in enumerate(chars)}
print(char2ix)
print(ix2char)

{'ü': 0, 'G': 1, 'W': 2, '.': 3, 'N': 4, 'ô': 5, 'V': 6, 's': 7, '{': 8, 'Y': 9, 'ñ': 10, '-': 11, 'O': 12, ':': 13, '[': 14, '¡': 15, 'B': 16, 'L': 17, ',': 18, '3': 19, 'r': 20, 'Á': 21, '\n': 22, 'p': 23, 'é': 24, 'x': 25, '9': 26, 'æ': 27, 'm': 28, 'Í': 29, 'a': 30, 'h': 31, '}': 32, 'D': 33, '*': 34, 'É': 35, 'K': 36, 'â': 37, 'E': 38, 'I': 39, "'": 40, 'J': 41, '(': 42, 'q': 43, ']': 44, 'R': 45, 'T': 46, '0': 47, '¶': 48, 'i': 49, 'd': 50, 'H': 51, '4': 52, 'ö': 53, 'y': 54, 'M': 55, 'j': 56, '6': 57, '¿': 58, 'U': 59, '_': 60, ';': 61, 'ú': 62, 'c': 63, '7': 64, 'Ú': 65, 'í': 66, 'z': 67, 'Q': 68, 'á': 69, 'b': 70, 'ó': 71, 'S': 72, 'g': 73, 'v': 74, '2': 75, '5': 76, '!': 77, 'f': 78, 'w': 79, 'u': 80, 'o': 81, '8': 82, '?': 83, ' ': 84, '"': 85, 'F': 86, 'l': 87, 'Ó': 88, '1': 89, 'X': 90, 'n': 91, 't': 92, ')': 93, 'e': 94, 'A': 95, 'C': 96, 'P': 97, 'Ö': 98, 'k': 99}
{0: 'ü', 1: 'G', 2: 'W', 3: '.', 4: 'N', 5: 'ô', 6: 'V', 7: 's', 8: '{', 9: 'Y', 10: 'ñ', 11: '-', 12: 'O', 

In [ ]:
# Crear bloques de 10 caracteres, que serán los datos de entrada a la red
LONG_SEC = 10
texto_in, texto_out = [], []

# Por cada 10 caracteres de entrada, el modelo predice 1 caracter de salida
for i in range(0,len(texto)-LONG_SEC):
  texto_in.append(texto[i:i+LONG_SEC]) # Ejemplo: caracteres del 0 al 9
  texto_out.append(texto[i + LONG_SEC])# Ejemplo: caracter 10

print(texto_in[0])
print(texto_out[0])
print(texto_in[1])
print(texto_out[1])

EL BARRIL 
D
L BARRIL D
E


In [ ]:
# Set de entrenamiento: simplemente se deben convertir a one-hot los sets
# de entrada y salida

X = np.zeros((len(texto_in), LONG_SEC, tam_vocab)) # #ejemplos x 10 x tam_vocab
Y = np.zeros((len(texto_in), tam_vocab))   # #ejemplos x tam_vocab

for i, entrada in enumerate(texto_in):
  for j, car in enumerate(entrada):
    X[i,j, char2ix[car]] = 1
    Y[i, char2ix[texto_out[i]]] = 1

print(X.shape)
print(Y.shape)
print(X[0,0,:])
print(Y[0,:])

(331185, 10, 100)
(331185, 100)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


## 3 - Crear modelo

In [ ]:
SEED = 15
tf.random.set_seed(SEED)
np.random.seed(SEED)
N_NEURONAS = 256

modelo = Sequential()
modelo.add( SimpleRNN(N_NEURONAS, input_shape=(LONG_SEC, tam_vocab)) )
modelo.add(Dense(tam_vocab, activation='softmax'))

modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 256)               91392     
_________________________________________________________________
dense (Dense)                (None, 100)               25700     
Total params: 117,092
Trainable params: 117,092
Non-trainable params: 0
_________________________________________________________________


## 4 - Función para generación de texto

In [ ]:
def generar_texto(modelo, L_OUT_SEC):
  # Seleccionar un dato de entrenamiento inicial aleatoriamente
  ix_test = np.random.randint(len(texto_in))
  char_test = texto_in[ix_test]

  # Generar texto de tamaño L_OUT_SEC
  print(char_test, end="")
  for i in range(L_OUT_SEC):
    # Codificación one-hot del texto de entrada
    X_test = np.zeros((1, LONG_SEC, tam_vocab)) # 1x10x100
    for j, car in enumerate(char_test):
      X_test[0,j, char2ix[car]] = 1
    
    # Introducir al modelo y generar predicción
    pred = modelo.predict(X_test, verbose=0)[0]
    y_pred = ix2char[np.argmax(pred)]

    # Imprimir el caracter predicho
    print(y_pred, end="")

    # Agregar como último elemento de "char_test" y continuar prediciendo
    char_test = char_test[1:] + y_pred
  print()

## 5 - Entrenamiento y generación de texto

In [ ]:
modelo.compile(loss="categorical_crossentropy", optimizer="rmsprop")

NEPOCHS = 20
BATCH_SIZE = 128
L_OUT_SEC = 50      # Número de caracteres a generar

print('-'*50)
print('Antes del entrenamiento:')
generar_texto(modelo, L_OUT_SEC)

for it in range(NEPOCHS):
  modelo.fit(X,Y, batch_size=BATCH_SIZE, epochs=1, verbose=0)
  print('-'*50)
  print(f'Iteración: {it+1}')
  generar_texto(modelo, L_OUT_SEC)

--------------------------------------------------
Antes del entrenamiento:
aba y acarôô0s¡¿EH!ivdúéykXYLó73.M,1ÖVylndAId¶üÁ:zwí_gO"ÍÓLi
--------------------------------------------------
Iteración: 1
 ruin amista de la de la de la de la de la de la de la de la
--------------------------------------------------
Iteración: 2
 doble Dupor en la de la cante en se había esta de la cante 
--------------------------------------------------
Iteración: 3
eza, mucho de la cama en la camo en la había esta abre la ca
--------------------------------------------------
Iteración: 4
 pausada y de su había de la casa en la vista de la casa en 
--------------------------------------------------
Iteración: 5
s. Legrande a des los por el mino de la casa de aprose de al
--------------------------------------------------
Iteración: 6
encontrába la casa de acrones de la casa de acrones de la ca
--------------------------------------------------
Iteración: 7
respecto?--exclame a dista de la casa de acrones 